In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, r2_score

%matplotlib inline

In [6]:
data = pd.read_csv('train.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 31, saw 3


In [ ]:
data

In [ ]:
data.describe(include=[object])['MSZoning']['top']

In [ ]:
data['GrLivArea'].describe()['max']

In [ ]:
data.describe(include=[object])['LotShape']

In [ ]:
plt.plot(data['GrLivArea'], data['SalePrice'], 'ro')

In [ ]:
def cleaning(data):
    categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
    numerical_columns   = [c for c in data.columns if (data[c].dtype.name != 'object' and c != 'SalePrice')]
    answer_column = [c for c in data.columns if c == 'SalePrice']
    #заполняем пустые количественные медианным значением
    data = data.fillna(data.median(axis=0), axis=0) 
    #заполняем пустые категориальные самым частым значением по признаку
    data_describe = data.describe(include=[object])
    for c in categorical_columns:                   
        data[c] = data[c].fillna(data_describe[c]['top'])
    #начинаем векторизацию - переводим категориальные признаки в количественные
    binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2] #бинарные
    nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2] #небинарные
    for c in binary_columns:
        top = data_describe[c]['top']
        top_items = data[c] == top
        data.loc[top_items, c] = 0
        data.loc[np.logical_not(top_items), c] = 1
    data_nonbinary = pd.get_dummies(data[nonbinary_columns])
    #начинаем нормализацию количественных признаков
    data_numerical = data[numerical_columns]
    data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
    data_answer = data[answer_column] #не требуется нормализация
    #соединяем всё в таблицу
    data = pd.concat((data_numerical, data[binary_columns], data_nonbinary, data_answer), axis=1)
    data = pd.DataFrame(data, dtype=float)
    return data

In [ ]:
data = cleaning(data)

In [ ]:
data['LotShape_IR1'].describe()

###### увеличилось число входных фич (!)

## Для нахождения наиболее информативных фич в общем случае можно применять алгоритм случайного леса и метод главных компонент. Но у нас есть описания каждой характеристики, поэтому можно повыбирать ручками:

In [ ]:
data.corr()

In [ ]:
data.corr()['SalePrice'].abs().sort_values(ascending=False)

### Теперь можно ковырять. Обратимся к data_description.txt:


#### OverallQual: общая оценка - понятно, сильнее всего коррелирует с ценой продажи
#### GrLivArea: жилая площадь самого дома - один из наиболее значимых параметров дома
#### GarageCars, GarageArea - значит для покупателя одно и то же. Второй признак уберём
#### TotalBsmtSF: общая площадь подвала дома - непонятно, насколько сильно коррелирует с GrLivArea. Проверяем:

In [ ]:
data['GrLivArea'].corr(data['TotalBsmtSF'])

#### сойдет
#### 1stFlrSF: площадь 1 этажа  - значим для покупателя почти так же, как и TotalBsmtSF. Уберём
#### ExterQual_ТА: качество внешней отделки (среднее) - сильно коррелирует с ценой, важный признак
#### FullBath: полноценные ванные комнаты, находящиеся не в подвале - ну да, прикольно
#### BsmtQual_Ex: высота подвала (отличная, +100inch) - важно
#### TotRmsAbvGrd: кол-во комнат выше подвала - важно
#### YearBuilt: дата постройки - важно
#### KitchenQual_TA: качество кухни (среднее) - важно
#### GarageFinish_Unf: качество интерьера гаража (незаконченное) - людям нравится, но он имеет отрицательную корреляцию с кол-вом машин в гараже - под вопросом ???
#### YearRemodAdd: ремонт - слишком сильно коррелирует с датой постройки. Уберём
#### KitchenQual_Ex: не сильно коррелирует со значением TA, оставим
### То, что имеет корреляцию с ценой продажи ниже 0.5, учитывать не будем
### Это не самый эффективный способ с точки зрения точности, но он эффективен с точки зрения возможности проанализировать исходный датасет

In [ ]:
data = data[['OverallQual','GrLivArea','GarageCars','TotalBsmtSF','ExterQual_TA','FullBath','BsmtQual_Ex','TotRmsAbvGrd','YearBuilt','KitchenQual_TA','GarageFinish_Unf','KitchenQual_Ex','SalePrice']]

In [ ]:
plt.plot(data['GrLivArea'], data['SalePrice'], 'ro')

### Устраняем выбросы:

In [ ]:
data=data[(np.abs(stats.zscore(data)) < 3).all(axis=1)] # красиво

In [ ]:
plt.plot(data['GrLivArea'], data['SalePrice'], 'ro')

In [ ]:
x = data.drop(('SalePrice'), axis=1)  # входные фичи
y = data['SalePrice']                 # ответ

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.5, random_state = 11)

### Задудосим линейную регрессию:

In [ ]:
regr = linear_model.LinearRegression(fit_intercept=True)
regr.fit(x_train, y_train)
y_valid_predict = regr.predict(x_valid)

In [ ]:
print('Coefficients: \n', regr.coef_)
print("MAE: %.2f" % mean_absolute_error(y_valid, y_valid_predict))
print('Variance score: %.2f' % r2_score(y_valid, y_valid_predict))

### Подключим тестовый датасет:

In [ ]:
x_test = pd.read_csv('./KaggleLab3/test.csv')
y_test = pd.read_csv('./KaggleLab3/sample_submission.csv')

### Подготавливаем к анализу:

In [ ]:
x_test = cleaning(x_test)
x_test = x_test[['OverallQual','GrLivArea','GarageCars','TotalBsmtSF','ExterQual_TA','FullBath','BsmtQual_Ex','TotRmsAbvGrd','YearBuilt','KitchenQual_TA','GarageFinish_Unf','KitchenQual_Ex']]
y_test = y_test[['SalePrice']]

### Делаем предсказание и сравниваем с ответами:

In [ ]:
y_test_predict = regr.predict(x_test)
print('Coefficients: \n', regr.coef_)
print("MAE: %.2f" % mean_absolute_error(y_test, y_test_predict))
print('Variance score: %.2f' % r2_score(y_test, y_test_predict))

### Мы получили большую среднюю ошибку, это связано с неидеальным отбором параметров. Но она ниже, чем в готовом примере на гитхабе лабораторной работы - там тупо взяли 3 параметра.

### Заливаем в .csv

In [ ]:
predictions = pd.DataFrame({'SalePrice':y_test_predict.round()}, index=range(1461,2920))
predictions.index.name = 'Id'

In [ ]:
predictions.to_csv('./KaggleLab3/predictions.csv')